In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter


import pickle
import gc

from tqdm import tqdm
tqdm.pandas()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

In [3]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = './models'
UTILS = './utils'

## prepare data, test / training sets

In [4]:
data = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp'])
data.head()

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen


In [5]:
def get_time_of_day(inp_hour):
    if (inp_hour >= 12) and (inp_hour < 18):
        return 'day'
    elif (inp_hour >= 6) and (inp_hour < 12):
        return 'morning'
    elif (inp_hour >= 18) and (inp_hour <= 23):
        return 'evening'
    else:
        return 'night'

In [6]:
%%time
data['dom']  = data.timestamp.apply(lambda x: x.day)
data['dow']  = data.timestamp.apply(lambda x: x.weekday())
data['hour'] = data.timestamp.apply(lambda x: x.hour)
data['tod']  = data.hour.apply(get_time_of_day)

data.head()

Wall time: 49.6 s


,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,15,0,14,day
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,21,5,12,day
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,24,4,18,evening
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,15,6,19,evening
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,20,3,7,morning


In [7]:
data.keys()[4:]

Index(['dom', 'dow', 'hour', 'tod'], dtype='object')

In [8]:
#target = data['multi_class_target']

In [9]:
#lb_dom  = LabelBinarizer().fit(data['dom'])
#lb_dow  = LabelBinarizer().fit(data['dow'])
#lb_hour = LabelBinarizer().fit(data['hour'])
#lb_tod  = LabelBinarizer().fit(data['tod'])

In [10]:
#dom_features  = ['dom_'  + str(el) for el in lb_dom.classes_]
#dow_features  = ['dow_'  + str(el) for el in lb_dow.classes_]
#hour_features = ['hour_' + str(el) for el in lb_hour.classes_]
#tod_features  = ['tod_'  + str(el) for el in lb_tod.classes_]

In [11]:
#%%time
#dom_prep  = lb_dom.transform(data['dom'])
#dow_prep  = lb_dow.transform(data['dow'])
#hour_prep = lb_hour.transform(data['hour'])
#tod_prep =  lb_tod.transform(data['tod'])

#dom_prep.shape, dow_prep.shape, hour_prep.shape, tod_prep.shape

saving LabelBinarizer for using in prepare to submit part

In [12]:
#pickle.dump(lb_dom,  open((os.path.join(UTILS, 'lb_dom.pkl')),  'wb'))
#pickle.dump(lb_dow,  open((os.path.join(UTILS, 'lb_dow.pkl')),  'wb'))
#pickle.dump(lb_hour, open((os.path.join(UTILS, 'lb_hour.pkl')), 'wb'))
#pickle.dump(lb_tod,  open((os.path.join(UTILS, 'lb_tod.pkl')),  'wb'))

## merge current data with client statistics

In [13]:
%%time 
client_freq_targ = pd.read_csv(os.path.join(DATA_OWN, 'client_freq.csv'))
client_diff_freq_dow = pd.read_csv(os.path.join(DATA_OWN, 'client_diff_freq_dow.csv'))
client_diff_freq_tod = pd.read_csv(os.path.join(DATA_OWN, 'client_diff_freq_tod.csv'))

Wall time: 1.28 s


In [14]:
col = ['client_pin', 'dow']
col.extend(['dow_'+el for el in client_diff_freq_dow.keys()[2:]])
client_diff_freq_dow.columns = col


col = ['client_pin', 'tod']
col.extend(['tod_'+el for el in client_diff_freq_tod.keys()[2:]])
client_diff_freq_tod.columns = col
client_diff_freq_dow.keys(), client_diff_freq_tod.keys()

(Index(['client_pin', 'dow', 'dow_diff_client_freq_main_screen',
        'dow_diff_client_freq_statement', 'dow_diff_client_freq_credit_info',
        'dow_diff_client_freq_own_transfer',
        'dow_diff_client_freq_mobile_recharge',
        'dow_diff_client_freq_phone_money_transfer',
        'dow_diff_client_freq_card2card_transfer', 'dow_diff_client_freq_chat',
        'dow_diff_client_freq_card_recharge', 'dow_diff_client_freq_invest'],
       dtype='object'),
 Index(['client_pin', 'tod', 'tod_diff_client_freq_main_screen',
        'tod_diff_client_freq_statement', 'tod_diff_client_freq_credit_info',
        'tod_diff_client_freq_own_transfer',
        'tod_diff_client_freq_mobile_recharge',
        'tod_diff_client_freq_phone_money_transfer',
        'tod_diff_client_freq_card2card_transfer', 'tod_diff_client_freq_chat',
        'tod_diff_client_freq_card_recharge', 'tod_diff_client_freq_invest'],
       dtype='object'))

In [15]:
data = data.merge(client_freq_targ, how= 'left', on='client_pin', validate='many_to_one')
data.shape

In [16]:
#client_freq_features = ['client_freq_main_screen', 'client_freq_statement',
#       'client_freq_credit_info', 'client_freq_own_transfer',
#       'client_freq_mobile_recharge', 'client_freq_phone_money_transfer',
#       'client_freq_card2card_transfer', 'client_freq_chat',
#       'client_freq_card_recharge', 'client_freq_invest']
#client_freq_features = client_freq_targ.keys()[1:]

In [17]:
#data.head()

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,client_freq_chat,client_freq_card_recharge,client_freq_invest
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,15,0,14,day,0.133333,0.380952,0.000000,0.000000,0.066667,0.142857,0.276190,0.000000,0.000000,0.000000
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,21,5,12,day,0.389706,0.573529,0.000000,0.007353,0.000000,0.000000,0.000000,0.022059,0.000000,0.007353
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,24,4,18,evening,0.116279,0.465116,0.000000,0.046512,0.279070,0.069767,0.000000,0.023256,0.000000,0.000000
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,15,6,19,evening,0.457831,0.449799,0.000000,0.004016,0.032129,0.008032,0.024096,0.012048,0.004016,0.008032
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,20,3,7,morning,0.600000,0.066667,0.266667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000


In [18]:
data = pd.merge(data, client_diff_freq_dow,  how='left', left_on=['client_pin', 'dow'], right_on = ['client_pin','dow'])
data.shape

In [19]:
#data.head()

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,client_freq_main_screen,client_freq_statement,...,dow_diff_client_freq_main_screen,dow_diff_client_freq_statement,dow_diff_client_freq_credit_info,dow_diff_client_freq_own_transfer,dow_diff_client_freq_mobile_recharge,dow_diff_client_freq_phone_money_transfer,dow_diff_client_freq_card2card_transfer,dow_diff_client_freq_chat,dow_diff_client_freq_card_recharge,dow_diff_client_freq_invest
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,15,0,14,day,0.133333,0.380952,...,0.008333,0.193452,0.000000,0.000000,0.004167,0.017857,-0.223810,0.000000,0.000000,0.000000
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,21,5,12,day,0.389706,0.573529,...,-0.022059,0.044118,0.000000,0.007353,0.000000,0.000000,0.000000,0.022059,0.000000,-0.051471
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,24,4,18,evening,0.116279,0.465116,...,0.116279,-0.368217,0.000000,-0.036822,0.195736,0.069767,0.000000,0.023256,0.000000,0.000000
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,15,6,19,evening,0.457831,0.449799,...,-0.226379,0.291904,0.000000,-0.048616,0.032129,0.008032,-0.028535,-0.040583,0.004016,0.008032
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,20,3,7,morning,0.600000,0.066667,...,0.100000,0.066667,-0.233333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000


In [20]:
data = pd.merge(data, client_diff_freq_tod,  how='left', left_on=['client_pin', 'tod'], right_on = ['client_pin','tod'])
data.shape

(5065350, 38)

In [21]:
#new_df = pd.merge(A_df, B_df,  how='left', left_on='[A_c1,c2]', right_on = '[B_c1,c2]')

## merge current data with global statistics

In [23]:
glob_diff_freq_dom  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dom.csv'))
glob_diff_freq_dow  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dow.csv'))
glob_diff_freq_hour = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_hour.csv'))
glob_diff_freq_tod  = pd.read_csv(os.path.join(DATA_OWN, 'glob_diff_freq_tod.csv'))

In [24]:
glob_diff_freq_dom.columns  = ['dom_'  + el for el in glob_diff_freq_dom.keys()]
glob_diff_freq_dow.columns  = ['dow_'  + el for el in glob_diff_freq_dow.keys()]
glob_diff_freq_hour.columns = ['hour_' + el for el in glob_diff_freq_hour.keys()]
glob_diff_freq_tod.columns  = ['tod_'  + el for el in glob_diff_freq_tod.keys()]

In [25]:
glob_diff_freq_dom = glob_diff_freq_dom.rename(  columns={'dom_dom': 'dom'})
glob_diff_freq_dow = glob_diff_freq_dow.rename(  columns={'dow_dow': 'dow'})
glob_diff_freq_hour = glob_diff_freq_hour.rename(columns={'hour_hour': 'hour'})
glob_diff_freq_tod = glob_diff_freq_tod.rename(  columns={'tod_tod': 'tod'})

#glob_freq_dom.keys(), glob_freq_dow.keys(), glob_freq_hour.keys(), glob_freq_tod.keys(), 

In [26]:
dom_diff_freq_features  = [el for el in glob_diff_freq_dom.keys()[1:]]
dow_diff_freq_features  = [el for el in glob_diff_freq_dow.keys()[1:]]
hour_diff_freq_features = [el for el in glob_diff_freq_hour.keys()[1:]]
tod_diff_freq_features  = [el for el in glob_diff_freq_tod.keys()[1:]]

In [27]:
data.shape

(5065350, 38)

In [28]:
data = data.merge(glob_diff_freq_dom, how= 'left', on='dom', validate='many_to_one')
data.shape

(5065350, 48)

In [29]:
data = data.merge(glob_diff_freq_dow, how= 'left', on='dow', validate='many_to_one')
data.shape

(5065350, 58)

In [30]:
data = data.merge(glob_diff_freq_hour, how= 'left', on='hour', validate='many_to_one')
data.shape

(5065350, 68)

In [31]:
data = data.merge(glob_diff_freq_tod, how= 'left', on='tod', validate='many_to_one')
data.shape

(5065350, 78)

In [32]:
data.sample(10)

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,client_freq_main_screen,client_freq_statement,...,tod_diff_glob_freq_main_screen,tod_diff_glob_freq_statement,tod_diff_glob_freq_credit_info,tod_diff_glob_freq_own_transfer,tod_diff_glob_freq_mobile_recharge,tod_diff_glob_freq_phone_money_transfer,tod_diff_glob_freq_card2card_transfer,tod_diff_glob_freq_chat,tod_diff_glob_freq_card_recharge,tod_diff_glob_freq_invest
785514,bf5e3588ba61a5fbef38062d3a823760,faa1246ed8285734b897c901d420c57a,2020-08-26 14:39:47,phone_money_transfer,26,2,14,day,0.618321,0.091603,...,-0.006382,-0.003040,0.008693,-0.003441,0.010104,-0.003059,-0.002092,-0.000810,-0.001044,0.001070
3110251,843982312af414ac6a47341322e53566,a6ea5a85b701a8574908b874ed81242a,2020-07-20 19:59:00,main_screen,20,0,19,evening,0.828042,0.047619,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
3258115,d346cb2304f7f9734201382e96d31b67,ec353db18ba30e03d51c5dda0b8645cf,2020-01-15 20:25:30,credit_info,15,2,20,evening,0.211538,0.019231,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
818663,5f7f329c5c5ee8c3ac8a0e452e967650,79bb572e625ec0d6dba2bdb9eed36bad,2020-05-28 11:49:08,main_screen,28,3,11,morning,0.309322,0.516949,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
2356154,462c52e20e68ba29ac6c80e5bf49af3e,4a1253dc81abfdb8f2e03d4ff00dad71,2020-08-11 09:45:23,statement,11,1,9,morning,0.150794,0.071429,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
2509077,d2ec87646518929ca0fd8607f32543ce,726ad13c8275bc3b4bb00aa0cbbbfa42,2020-08-07 08:30:28,credit_info,7,4,8,morning,0.052632,0.000000,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
2106008,e78d74a1d3556342aced3562dd066c0a,e19fd65b9589ebbe3476be224ceb2b67,2020-01-30 18:28:45,main_screen,30,3,18,evening,0.452830,0.339623,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486
3554852,70368912655a02c07826a1e1e4264575,8169b1a62eb3ee10028208835a0acad7,2020-07-22 05:38:02,main_screen,22,2,5,night,0.618625,0.039911,...,0.002702,0.004453,-0.022015,0.011315,-0.018217,0.011603,0.006227,0.002242,0.003320,-0.001628
3940002,7752a55cc56e1540fb2be91bf866134e,ec717d740b5c76a99a9dda7d96382491,2020-03-01 11:11:07,main_screen,1,6,11,morning,0.728086,0.228980,...,-0.007052,0.016034,-0.004793,0.002914,-0.013680,0.000881,0.003722,0.001396,0.000394,0.000184
4160718,5b59be9d1f2e8859cda4dff9caf7591d,4769d09e7f4eabb4857e1d4dd9f00fee,2020-02-25 20:36:29,statement,25,1,20,evening,0.213043,0.373913,...,0.018425,-0.016537,-0.001558,-0.001434,0.006193,0.000416,-0.003130,-0.001105,0.000214,-0.001486


In [ ]:
#f1 = make_scorer(f1_score , average='macro')

### create using features list

In [39]:
using_features = data.keys()[8:]
#print(using_features)

## saving

In [42]:
%%time
data.to_csv(os.path.join(DATA_OWN, 'data_train.csv'))

Wall time: 8min 47s


In [43]:
%%time
pickle.dump(using_features, open(os.path.join(DATA_OWN, 'using_features.pkl'), 'wb'))

Wall time: 0 ns
